<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

## Lab 4.2.2: Feature Selection

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
%matplotlib inline

### 5. Forward Feature Selection

> Forward Selection: Forward selection is an iterative method in which we start with having no feature in the model. In each iteration, we keep adding the feature which best improves our model till an addition of a new variable does not improve the performance of the model.

Create a Regression model using Forward Feature Selection by looping over all the features adding one at a time until there are no improvements on the prediction metric ( R2  and  AdjustedR2  in this case).

#### 5.1 Load Wine Data & Define Predictor and Target

In [3]:
## Load the wine quality dataset

# Load the wine dataset from csv
wine = pd.read_csv('data/winequality_merged.csv')

# define the target variable (dependent variable) as y
y = wine['quality']

# Take all columns except target as predictor columns
predictor_columns = [c for c in wine.columns if c != 'quality']
# Load the dataset as a pandas data frame
X = pd.DataFrame(wine, columns = predictor_columns)

In [4]:
X

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,red_wine
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,1
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,1
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,1
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,1
...,...,...,...,...,...,...,...,...,...,...,...,...
6492,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,0
6493,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,0
6494,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,0
6495,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,0


In [5]:
## Create training and testing subsets

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [6]:
y_train

355     6
2405    5
1483    5
622     5
4097    5
       ..
5594    5
5072    8
6046    6
5288    8
2469    5
Name: quality, Length: 5197, dtype: int64

#### 5.2 Overview of the code below

The external `while` loop goes forever until there are no improvements to the model, which is controlled by the flag `changed` (until is **not** changed).
The inner `for` loop goes over each of the features not yet included in the model and calculates the correlation coefficient. If any model improves on the previous best model then the records are updated.

#### Code variables
- `included`: list of the features (predictors) that were included in the model; starts empty.
- `excluded`: list of features that have **not** been included in the model; starts as the full list of features.
- `best`: dictionary to keep record of the best model found at any stage; starts 'empty'.
- `model`: object of class LinearRegression, with default values for all parameters.

#### Methods of the `LinearRegression` object to investigate
- `fit()`
- `fit.score()`

#### Adjusted $R^2$ formula
$$Adjusted \; R^2 = 1 - { (1 - R^2) (n - 1)  \over n - k - 1 }$$

#### Linear Regression [reference](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html#sklearn.linear_model.LinearRegression)

In [7]:
## Flag intermediate output

show_steps = True   # for testing/debugging
# show_steps = False  # without showing steps

# start with no predictors
included = []

In [8]:
## Use Forward Feature Selection to pick a good model


# keep track of model and parameters
best = {'feature': '', 'r2': 0, 'a_r2': 0}
# create a model object to hold the modelling parameters
model = LinearRegression() # create a model for Linear Regression
# get the number of cases in the training data
n = X_train.shape[0]

while True:
    changed = False
    
    if show_steps:
        print('') 

    # list the features to be evaluated
    excluded = list(set(X.columns) - set(included))
    
    if show_steps:
        print('(Step) Excluded = %s' % ', '.join(excluded))  

    # for each remaining feature to be evaluated
    for new_column in excluded:
        
        if show_steps:
            print('(Step) Trying %s...' % new_column)
            print('(Step) - Features = %s' % ', '.join(included + [new_column]))

        # fit the model with the Training data

        # setting training data
        fit = model.fit(X_train[included + [new_column]], y_train) # fit a model; 
        
        # calculate the score (R^2 for Regression)
        from sklearn.metrics import mean_squared_error, r2_score
        y_pred = model.predict(X_test[included + [new_column]])
        r2 = fit.score(X_train[included + [new_column]], y_train) # calculate the score
        
        # calculate the adjusted R^2
        k_ = len(included + [new_column])
        adjusted_r2 = 1 - ( ( (1 - r2) * (n - 1) ) / (n - k_ - 1) ) # calculate the Adjusted R^2

        if show_steps:
            print('(Step) - Adjusted R^2: This = %.3f; Best = %.3f' % 
                  (adjusted_r2, best['a_r2']))

        # if model improves
        if adjusted_r2 > best['a_r2']:
            # record new parameters
            best = {'feature': new_column, 'r2': r2, 'a_r2': adjusted_r2}
            # flag that found a better model
            changed = True
            if show_steps:
                print('(Step) - New Best!   : Feature = %s; R^2 = %.3f; Adjusted R^2 = %.3f' % 
                      (best['feature'], best['r2'], best['a_r2']))
    # END for

    # if found a better model after testing all remaining features
    if changed:
        # update control details
        included.append(best['feature'])
#         print('included' , included)
        excluded = list(set(excluded) - set(best['feature']))
        print('Added feature %-4s with R^2 = %.3f and adjusted R^2 = %.3f' % 
              (best['feature'], best['r2'], best['a_r2']))
    else:
        # terminate if no better model
        break

print('')
print('Resulting features:')
print(', '.join(included))


(Step) Excluded = sulphates, fixed acidity, citric acid, red_wine, chlorides, density, total sulfur dioxide, alcohol, free sulfur dioxide, volatile acidity, pH, residual sugar
(Step) Trying sulphates...
(Step) - Features = sulphates
(Step) - Adjusted R^2: This = 0.002; Best = 0.000
(Step) - New Best!   : Feature = sulphates; R^2 = 0.002; Adjusted R^2 = 0.002
(Step) Trying fixed acidity...
(Step) - Features = fixed acidity
(Step) - Adjusted R^2: This = 0.007; Best = 0.002
(Step) - New Best!   : Feature = fixed acidity; R^2 = 0.007; Adjusted R^2 = 0.007
(Step) Trying citric acid...
(Step) - Features = citric acid
(Step) - Adjusted R^2: This = 0.007; Best = 0.007
(Step) - New Best!   : Feature = citric acid; R^2 = 0.007; Adjusted R^2 = 0.007
(Step) Trying red_wine...
(Step) - Features = red_wine
(Step) - Adjusted R^2: This = 0.016; Best = 0.007
(Step) - New Best!   : Feature = red_wine; R^2 = 0.016; Adjusted R^2 = 0.016
(Step) Trying chlorides...
(Step) - Features = chlorides
(Step) - Ad

(Step) - Adjusted R^2: This = 0.288; Best = 0.284
(Step) - New Best!   : Feature = density; R^2 = 0.288; Adjusted R^2 = 0.288
(Step) Trying total sulfur dioxide...
(Step) - Features = alcohol, volatile acidity, sulphates, residual sugar, red_wine, total sulfur dioxide
(Step) - Adjusted R^2: This = 0.283; Best = 0.288
(Step) Trying free sulfur dioxide...
(Step) - Features = alcohol, volatile acidity, sulphates, residual sugar, red_wine, free sulfur dioxide
(Step) - Adjusted R^2: This = 0.287; Best = 0.288
(Step) Trying pH...
(Step) - Features = alcohol, volatile acidity, sulphates, residual sugar, red_wine, pH
(Step) - Adjusted R^2: This = 0.284; Best = 0.288
Added feature density with R^2 = 0.288 and adjusted R^2 = 0.288

(Step) Excluded = fixed acidity, citric acid, chlorides, total sulfur dioxide, free sulfur dioxide, pH
(Step) Trying fixed acidity...
(Step) - Features = alcohol, volatile acidity, sulphates, residual sugar, red_wine, density, fixed acidity
(Step) - Adjusted R^2: This



---



---



> > > > > > > > > © 2021 Institute of Data


---



---



